In [4]:
#les imports nécessaires 
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

# Create training and evaluation datasets

In [5]:
def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

# Compile, train, and evaluate the model

In [6]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

# Create model inputs

In [7]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs

# Experiment 1: standard neural network

In [8]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [9]:
#split the wine dataset into training and test sets, with 85% and 15% of the examples, respectively
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\SOUFIANI Assia\tensorflow_datasets\wine_quality\white\1.0.0.incompleteSYEEF7\wine_quality-t…

Dataset wine_quality downloaded and prepared to C:\Users\SOUFIANI Assia\tensorflow_datasets\wine_quality\white\1.0.0. Subsequent calls will reuse this data.


In [10]:
#train the baseline model. We use the MeanSquaredError as the loss function
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/100
17/17 [==============================] - 5s 89ms/step - loss: 30.9575 - root_mean_squared_error: 5.5639 - val_loss: 28.0353 - val_root_mean_squared_error: 5.2948
Epoch 2/100
17/17 [==============================] - 0s 10ms/step - loss: 28.8413 - root_mean_squared_error: 5.3704 - val_loss: 26.8525 - val_root_mean_squared_error: 5.1819
Epoch 3/100
17/17 [==============================] - 0s 11ms/step - loss: 27.0263 - root_mean_squared_error: 5.1987 - val_loss: 25.5216 - val_root_mean_squared_error: 5.0519
Epoch 4/100
17/17 [==============================] - 0s 9ms/step - loss: 25.2661 - root_mean_squared_error: 5.0265 - val_loss: 24.1004 - val_root_mean_squared_error: 4.9092
Epoch 5/100
17/17 [==============================] - 0s 10ms/step - loss: 23.5368 - root_mean_squared_error: 4.8515 - val_loss: 22.6273 - val_root_mean_squared_error: 4.7568
Epoch 6/100
17/17 [==============================] - 0s 10ms/step - loss: 21.8469 - root_mean_squared_e

17/17 [==============================] - 0s 10ms/step - loss: 0.6329 - root_mean_squared_error: 0.7956 - val_loss: 0.6388 - val_root_mean_squared_error: 0.7992
Epoch 49/100
17/17 [==============================] - 0s 12ms/step - loss: 0.6287 - root_mean_squared_error: 0.7929 - val_loss: 0.6323 - val_root_mean_squared_error: 0.7952
Epoch 50/100
17/17 [==============================] - 0s 10ms/step - loss: 0.6204 - root_mean_squared_error: 0.7877 - val_loss: 0.6263 - val_root_mean_squared_error: 0.7914
Epoch 51/100
17/17 [==============================] - 0s 11ms/step - loss: 0.6156 - root_mean_squared_error: 0.7846 - val_loss: 0.6210 - val_root_mean_squared_error: 0.7881
Epoch 52/100
17/17 [==============================] - 0s 10ms/step - loss: 0.6107 - root_mean_squared_error: 0.7814 - val_loss: 0.6173 - val_root_mean_squared_error: 0.7857
Epoch 53/100
17/17 [==============================] - 0s 11ms/step - loss: 0.6101 - root_mean_squared_error: 0.7811 - val_loss: 0.6126 - val_root_me

17/17 [==============================] - 0s 10ms/step - loss: 0.5755 - root_mean_squared_error: 0.7586 - val_loss: 0.5700 - val_root_mean_squared_error: 0.7550
Epoch 96/100
17/17 [==============================] - 0s 11ms/step - loss: 0.5739 - root_mean_squared_error: 0.7576 - val_loss: 0.5685 - val_root_mean_squared_error: 0.7540
Epoch 97/100
17/17 [==============================] - 0s 10ms/step - loss: 0.5728 - root_mean_squared_error: 0.7568 - val_loss: 0.5690 - val_root_mean_squared_error: 0.7543
Epoch 98/100
17/17 [==============================] - 0s 10ms/step - loss: 0.5755 - root_mean_squared_error: 0.7586 - val_loss: 0.5678 - val_root_mean_squared_error: 0.7536
Epoch 99/100
17/17 [==============================] - 0s 10ms/step - loss: 0.5720 - root_mean_squared_error: 0.7563 - val_loss: 0.5680 - val_root_mean_squared_error: 0.7536
Epoch 100/100
17/17 [==============================] - 0s 11ms/step - loss: 0.5713 - root_mean_squared_error: 0.7558 - val_loss: 0.5668 - val_root_m

In [11]:
#We take a sample from the test set use the model to obtain predictions for them
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

Predicted: 5.6 - Actual: 5.0
Predicted: 5.9 - Actual: 5.0
Predicted: 6.1 - Actual: 5.0
Predicted: 6.1 - Actual: 6.0
Predicted: 5.1 - Actual: 5.0
Predicted: 6.2 - Actual: 7.0
Predicted: 5.4 - Actual: 5.0
Predicted: 6.1 - Actual: 5.0
Predicted: 6.2 - Actual: 6.0
Predicted: 6.2 - Actual: 7.0


# Experiment 2: Bayesian neural network (BNN)

In [12]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

In [13]:
#We use the tfp.layers.DenseVariational layer instead of the standard keras.layers.Dense layer in the neural network model
def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Train BNN with a small training subset

In [14]:
num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


Start training the model...
Epoch 1/500
5/5 [==============================] - 8s 560ms/step - loss: 41.4455 - root_mean_squared_error: 6.4368 - val_loss: 39.8542 - val_root_mean_squared_error: 6.3118
Epoch 2/500
5/5 [==============================] - 0s 29ms/step - loss: 33.9443 - root_mean_squared_error: 5.8252 - val_loss: 33.8590 - val_root_mean_squared_error: 5.8175
Epoch 3/500
5/5 [==============================] - 0s 27ms/step - loss: 34.6922 - root_mean_squared_error: 5.8886 - val_loss: 32.6121 - val_root_mean_squared_error: 5.7093
Epoch 4/500
5/5 [==============================] - 0s 27ms/step - loss: 33.5006 - root_mean_squared_error: 5.7866 - val_loss: 29.2739 - val_root_mean_squared_error: 5.4094
Epoch 5/500
5/5 [==============================] - 0s 22ms/step - loss: 32.5230 - root_mean_squared_error: 5.7016 - val_loss: 35.3633 - val_root_mean_squared_error: 5.9455
Epoch 6/500
5/5 [==============================] - 0s 28ms/step - loss: 35.3098 - root_mean_squared_error: 5.94

5/5 [==============================] - 0s 25ms/step - loss: 16.2077 - root_mean_squared_error: 4.0234 - val_loss: 17.2169 - val_root_mean_squared_error: 4.1477
Epoch 49/500
5/5 [==============================] - 0s 26ms/step - loss: 19.7305 - root_mean_squared_error: 4.4404 - val_loss: 17.1426 - val_root_mean_squared_error: 4.1385
Epoch 50/500
5/5 [==============================] - 0s 25ms/step - loss: 15.2734 - root_mean_squared_error: 3.9063 - val_loss: 13.1641 - val_root_mean_squared_error: 3.6260
Epoch 51/500
5/5 [==============================] - 0s 28ms/step - loss: 16.5062 - root_mean_squared_error: 4.0605 - val_loss: 14.6946 - val_root_mean_squared_error: 3.8308
Epoch 52/500
5/5 [==============================] - 0s 31ms/step - loss: 15.5589 - root_mean_squared_error: 3.9424 - val_loss: 15.2143 - val_root_mean_squared_error: 3.8982
Epoch 53/500
5/5 [==============================] - 0s 26ms/step - loss: 15.5433 - root_mean_squared_error: 3.9404 - val_loss: 14.4575 - val_root_me

Epoch 96/500
5/5 [==============================] - 0s 26ms/step - loss: 6.2099 - root_mean_squared_error: 2.4878 - val_loss: 5.4484 - val_root_mean_squared_error: 2.3284
Epoch 97/500
5/5 [==============================] - 0s 30ms/step - loss: 4.7181 - root_mean_squared_error: 2.1668 - val_loss: 4.4689 - val_root_mean_squared_error: 2.1076
Epoch 98/500
5/5 [==============================] - 0s 25ms/step - loss: 4.1543 - root_mean_squared_error: 2.0317 - val_loss: 3.7873 - val_root_mean_squared_error: 1.9390
Epoch 99/500
5/5 [==============================] - 0s 25ms/step - loss: 4.3473 - root_mean_squared_error: 2.0792 - val_loss: 4.2894 - val_root_mean_squared_error: 2.0652
Epoch 100/500
5/5 [==============================] - 0s 25ms/step - loss: 3.9459 - root_mean_squared_error: 1.9806 - val_loss: 4.5320 - val_root_mean_squared_error: 2.1236
Epoch 101/500
5/5 [==============================] - 0s 27ms/step - loss: 4.4315 - root_mean_squared_error: 2.0998 - val_loss: 4.5335 - val_root

...

Epoch 287/500
5/5 [==============================] - 0s 25ms/step - loss: 0.7548 - root_mean_squared_error: 0.8510 - val_loss: 0.7511 - val_root_mean_squared_error: 0.8479
Epoch 288/500
5/5 [==============================] - 0s 25ms/step - loss: 0.7574 - root_mean_squared_error: 0.8558 - val_loss: 0.6766 - val_root_mean_squared_error: 0.8039
Epoch 289/500
5/5 [==============================] - 0s 28ms/step - loss: 0.7198 - root_mean_squared_error: 0.8316 - val_loss: 0.6841 - val_root_mean_squared_error: 0.8073
Epoch 290/500
5/5 [==============================] - 0s 27ms/step - loss: 0.7097 - root_mean_squared_error: 0.8249 - val_loss: 0.6687 - val_root_mean_squared_error: 0.7991
Epoch 291/500
5/5 [==============================] - 0s 22ms/step - loss: 0.7514 - root_mean_squared_error: 0.8490 - val_loss: 0.7032 - val_root_mean_squared_error: 0.8211
Epoch 292/500
5/5 [==============================] - 0s 28ms/step - loss: 0.7267 - root_mean_squared_error: 0.8357 - val_loss: 0.6676 - val_

5/5 [==============================] - 0s 30ms/step - loss: 0.6709 - root_mean_squared_error: 0.7997 - val_loss: 0.6694 - val_root_mean_squared_error: 0.8001
Epoch 335/500
5/5 [==============================] - 0s 25ms/step - loss: 0.7364 - root_mean_squared_error: 0.8418 - val_loss: 0.7501 - val_root_mean_squared_error: 0.8486
Epoch 336/500
5/5 [==============================] - 0s 29ms/step - loss: 0.6604 - root_mean_squared_error: 0.7950 - val_loss: 0.7276 - val_root_mean_squared_error: 0.8334
Epoch 337/500
5/5 [==============================] - 0s 24ms/step - loss: 0.6971 - root_mean_squared_error: 0.8155 - val_loss: 0.6918 - val_root_mean_squared_error: 0.8133
Epoch 338/500
5/5 [==============================] - 0s 28ms/step - loss: 0.6758 - root_mean_squared_error: 0.8028 - val_loss: 0.6750 - val_root_mean_squared_error: 0.8000
Epoch 339/500
5/5 [==============================] - 0s 24ms/step - loss: 0.6452 - root_mean_squared_error: 0.7850 - val_loss: 0.6791 - val_root_mean_squa

Epoch 382/500
5/5 [==============================] - 0s 25ms/step - loss: 0.6876 - root_mean_squared_error: 0.8116 - val_loss: 0.6367 - val_root_mean_squared_error: 0.7802
Epoch 383/500
5/5 [==============================] - 0s 26ms/step - loss: 0.6851 - root_mean_squared_error: 0.8091 - val_loss: 0.6469 - val_root_mean_squared_error: 0.7848
Epoch 384/500
5/5 [==============================] - 0s 28ms/step - loss: 0.7025 - root_mean_squared_error: 0.8205 - val_loss: 0.6613 - val_root_mean_squared_error: 0.7960
Epoch 385/500
5/5 [==============================] - 0s 29ms/step - loss: 0.6643 - root_mean_squared_error: 0.7973 - val_loss: 0.6742 - val_root_mean_squared_error: 0.8038
Epoch 386/500
5/5 [==============================] - 0s 26ms/step - loss: 0.6753 - root_mean_squared_error: 0.8038 - val_loss: 0.6809 - val_root_mean_squared_error: 0.8053
Epoch 387/500
5/5 [==============================] - 0s 24ms/step - loss: 0.6280 - root_mean_squared_error: 0.7745 - val_loss: 0.6647 - val_

5/5 [==============================] - 0s 23ms/step - loss: 0.6808 - root_mean_squared_error: 0.8061 - val_loss: 0.6768 - val_root_mean_squared_error: 0.8041
Epoch 430/500
5/5 [==============================] - 0s 25ms/step - loss: 0.6615 - root_mean_squared_error: 0.7945 - val_loss: 0.6470 - val_root_mean_squared_error: 0.7880
Epoch 431/500
5/5 [==============================] - 0s 22ms/step - loss: 0.6597 - root_mean_squared_error: 0.7931 - val_loss: 0.6921 - val_root_mean_squared_error: 0.8156
Epoch 432/500
5/5 [==============================] - 0s 27ms/step - loss: 0.7028 - root_mean_squared_error: 0.8199 - val_loss: 0.6316 - val_root_mean_squared_error: 0.7748
Epoch 433/500
5/5 [==============================] - 0s 26ms/step - loss: 0.6976 - root_mean_squared_error: 0.8168 - val_loss: 0.6829 - val_root_mean_squared_error: 0.8052
Epoch 434/500
5/5 [==============================] - 0s 29ms/step - loss: 0.6629 - root_mean_squared_error: 0.7961 - val_loss: 0.6830 - val_root_mean_squa

Epoch 477/500
5/5 [==============================] - 0s 25ms/step - loss: 0.6580 - root_mean_squared_error: 0.7927 - val_loss: 0.6359 - val_root_mean_squared_error: 0.7787
Epoch 478/500
5/5 [==============================] - 0s 31ms/step - loss: 0.6660 - root_mean_squared_error: 0.7963 - val_loss: 0.6653 - val_root_mean_squared_error: 0.7949
Epoch 479/500
5/5 [==============================] - 0s 32ms/step - loss: 0.6608 - root_mean_squared_error: 0.7946 - val_loss: 0.6659 - val_root_mean_squared_error: 0.7962
Epoch 480/500
5/5 [==============================] - 0s 27ms/step - loss: 0.7172 - root_mean_squared_error: 0.8296 - val_loss: 0.6540 - val_root_mean_squared_error: 0.7896
Epoch 481/500
5/5 [==============================] - 0s 26ms/step - loss: 0.6635 - root_mean_squared_error: 0.7968 - val_loss: 0.6559 - val_root_mean_squared_error: 0.7923
Epoch 482/500
5/5 [==============================] - 0s 28ms/step - loss: 0.6440 - root_mean_squared_error: 0.7831 - val_loss: 0.6696 - val_

In [15]:
#we have trained a BNN model
def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )


compute_predictions(bnn_model_small)

Predictions mean: 5.5, min: 4.69, max: 6.01, range: 1.32 - Actual: 5.0
Predictions mean: 5.92, min: 5.0, max: 6.21, range: 1.21 - Actual: 5.0
Predictions mean: 6.05, min: 5.1, max: 6.26, range: 1.16 - Actual: 5.0
Predictions mean: 6.09, min: 5.4, max: 6.3, range: 0.9 - Actual: 6.0
Predictions mean: 5.0, min: 4.31, max: 5.55, range: 1.24 - Actual: 5.0
Predictions mean: 5.99, min: 5.28, max: 6.33, range: 1.05 - Actual: 7.0
Predictions mean: 5.24, min: 4.26, max: 5.84, range: 1.58 - Actual: 5.0
Predictions mean: 6.0, min: 5.17, max: 6.27, range: 1.1 - Actual: 5.0
Predictions mean: 6.03, min: 5.36, max: 6.41, range: 1.06 - Actual: 6.0
Predictions mean: 6.16, min: 5.38, max: 6.32, range: 0.94 - Actual: 7.0


# Train BNN with the whole training set

In [16]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

compute_predictions(bnn_model_full)

Start training the model...
Epoch 1/500
17/17 [==============================] - 8s 122ms/step - loss: 36.9711 - root_mean_squared_error: 6.0800 - val_loss: 35.8346 - val_root_mean_squared_error: 5.9858
Epoch 2/500
17/17 [==============================] - 0s 11ms/step - loss: 34.9899 - root_mean_squared_error: 5.9148 - val_loss: 35.8123 - val_root_mean_squared_error: 5.9839
Epoch 3/500
17/17 [==============================] - 0s 12ms/step - loss: 33.3305 - root_mean_squared_error: 5.7729 - val_loss: 31.2909 - val_root_mean_squared_error: 5.5933
Epoch 4/500
17/17 [==============================] - 0s 13ms/step - loss: 32.1701 - root_mean_squared_error: 5.6715 - val_loss: 30.8364 - val_root_mean_squared_error: 5.5527
Epoch 5/500
17/17 [==============================] - 0s 12ms/step - loss: 31.1048 - root_mean_squared_error: 5.5768 - val_loss: 30.8214 - val_root_mean_squared_error: 5.5513
Epoch 6/500
17/17 [==============================] - 0s 11ms/step - loss: 29.7941 - root_mean_squared

Epoch 48/500
17/17 [==============================] - 0s 11ms/step - loss: 0.7776 - root_mean_squared_error: 0.8765 - val_loss: 0.8773 - val_root_mean_squared_error: 0.9321
Epoch 49/500
17/17 [==============================] - 0s 12ms/step - loss: 0.7640 - root_mean_squared_error: 0.8688 - val_loss: 0.7846 - val_root_mean_squared_error: 0.8800
Epoch 50/500
17/17 [==============================] - 0s 12ms/step - loss: 0.7529 - root_mean_squared_error: 0.8617 - val_loss: 0.7993 - val_root_mean_squared_error: 0.8885
Epoch 51/500
17/17 [==============================] - 0s 13ms/step - loss: 0.7705 - root_mean_squared_error: 0.8724 - val_loss: 0.7758 - val_root_mean_squared_error: 0.8751
Epoch 52/500
17/17 [==============================] - 0s 11ms/step - loss: 0.7498 - root_mean_squared_error: 0.8601 - val_loss: 0.7804 - val_root_mean_squared_error: 0.8779
Epoch 53/500
17/17 [==============================] - 0s 12ms/step - loss: 0.7536 - root_mean_squared_error: 0.8624 - val_loss: 0.9313 

17/17 [==============================] - 0s 10ms/step - loss: 0.6569 - root_mean_squared_error: 0.8038 - val_loss: 0.6548 - val_root_mean_squared_error: 0.8020
Epoch 96/500
17/17 [==============================] - 0s 11ms/step - loss: 0.6700 - root_mean_squared_error: 0.8119 - val_loss: 0.6917 - val_root_mean_squared_error: 0.8246
Epoch 97/500
17/17 [==============================] - 0s 13ms/step - loss: 0.6733 - root_mean_squared_error: 0.8139 - val_loss: 0.6740 - val_root_mean_squared_error: 0.8141
Epoch 98/500
17/17 [==============================] - 0s 11ms/step - loss: 0.6632 - root_mean_squared_error: 0.8076 - val_loss: 0.6815 - val_root_mean_squared_error: 0.8197
Epoch 99/500
17/17 [==============================] - 0s 10ms/step - loss: 0.6690 - root_mean_squared_error: 0.8112 - val_loss: 0.6879 - val_root_mean_squared_error: 0.8226
Epoch 100/500
17/17 [==============================] - 0s 11ms/step - loss: 0.6679 - root_mean_squared_error: 0.8108 - val_loss: 0.6629 - val_root_m

17/17 [==============================] - 0s 11ms/step - loss: 0.6212 - root_mean_squared_error: 0.7803 - val_loss: 0.6754 - val_root_mean_squared_error: 0.8144
Epoch 143/500
17/17 [==============================] - 0s 13ms/step - loss: 0.6245 - root_mean_squared_error: 0.7828 - val_loss: 0.6139 - val_root_mean_squared_error: 0.7758
Epoch 144/500
17/17 [==============================] - 0s 11ms/step - loss: 0.6390 - root_mean_squared_error: 0.7918 - val_loss: 0.6158 - val_root_mean_squared_error: 0.7777
Epoch 145/500
17/17 [==============================] - 0s 12ms/step - loss: 0.6264 - root_mean_squared_error: 0.7842 - val_loss: 0.6351 - val_root_mean_squared_error: 0.7902
Epoch 146/500
17/17 [==============================] - 0s 11ms/step - loss: 0.6313 - root_mean_squared_error: 0.7869 - val_loss: 0.6270 - val_root_mean_squared_error: 0.7840
Epoch 147/500
17/17 [==============================] - 0s 13ms/step - loss: 0.6296 - root_mean_squared_error: 0.7859 - val_loss: 0.6265 - val_ro

17/17 [==============================] - 0s 11ms/step - loss: 0.6383 - root_mean_squared_error: 0.7910 - val_loss: 0.6272 - val_root_mean_squared_error: 0.7838
Epoch 190/500
17/17 [==============================] - 0s 12ms/step - loss: 0.6280 - root_mean_squared_error: 0.7845 - val_loss: 0.6370 - val_root_mean_squared_error: 0.7900
Epoch 191/500
17/17 [==============================] - 0s 12ms/step - loss: 0.6210 - root_mean_squared_error: 0.7799 - val_loss: 0.6030 - val_root_mean_squared_error: 0.7678
Epoch 192/500
17/17 [==============================] - 0s 12ms/step - loss: 0.6241 - root_mean_squared_error: 0.7819 - val_loss: 0.6260 - val_root_mean_squared_error: 0.7834
Epoch 193/500
17/17 [==============================] - 0s 11ms/step - loss: 0.6308 - root_mean_squared_error: 0.7857 - val_loss: 0.6200 - val_root_mean_squared_error: 0.7801
Epoch 194/500
17/17 [==============================] - 0s 12ms/step - loss: 0.6231 - root_mean_squared_error: 0.7810 - val_loss: 0.6065 - val_ro

...

17/17 [==============================] - 0s 12ms/step - loss: 0.6187 - root_mean_squared_error: 0.7777 - val_loss: 0.6046 - val_root_mean_squared_error: 0.7694
Epoch 284/500
17/17 [==============================] - 0s 10ms/step - loss: 0.6131 - root_mean_squared_error: 0.7739 - val_loss: 0.6042 - val_root_mean_squared_error: 0.7674
Epoch 285/500
17/17 [==============================] - 0s 11ms/step - loss: 0.6153 - root_mean_squared_error: 0.7758 - val_loss: 0.6364 - val_root_mean_squared_error: 0.7890
Epoch 286/500
17/17 [==============================] - 0s 12ms/step - loss: 0.6250 - root_mean_squared_error: 0.7819 - val_loss: 0.6041 - val_root_mean_squared_error: 0.7691
Epoch 287/500
17/17 [==============================] - 0s 13ms/step - loss: 0.6059 - root_mean_squared_error: 0.7695 - val_loss: 0.6389 - val_root_mean_squared_error: 0.7909
Epoch 288/500
17/17 [==============================] - 0s 13ms/step - loss: 0.6095 - root_mean_squared_error: 0.7722 - val_loss: 0.6035 - val_ro

17/17 [==============================] - 0s 14ms/step - loss: 0.6075 - root_mean_squared_error: 0.7703 - val_loss: 0.6018 - val_root_mean_squared_error: 0.7662
Epoch 331/500
17/17 [==============================] - 0s 13ms/step - loss: 0.5997 - root_mean_squared_error: 0.7655 - val_loss: 0.5994 - val_root_mean_squared_error: 0.7658
Epoch 332/500
17/17 [==============================] - 0s 11ms/step - loss: 0.6093 - root_mean_squared_error: 0.7715 - val_loss: 0.5998 - val_root_mean_squared_error: 0.7653
Epoch 333/500
17/17 [==============================] - 0s 12ms/step - loss: 0.6068 - root_mean_squared_error: 0.7697 - val_loss: 0.5981 - val_root_mean_squared_error: 0.7645
Epoch 334/500
17/17 [==============================] - 0s 11ms/step - loss: 0.6080 - root_mean_squared_error: 0.7707 - val_loss: 0.6154 - val_root_mean_squared_error: 0.7758
Epoch 335/500
17/17 [==============================] - 0s 12ms/step - loss: 0.6090 - root_mean_squared_error: 0.7717 - val_loss: 0.6100 - val_ro

17/17 [==============================] - 0s 12ms/step - loss: 0.6033 - root_mean_squared_error: 0.7673 - val_loss: 0.5980 - val_root_mean_squared_error: 0.7645
Epoch 378/500
17/17 [==============================] - 0s 13ms/step - loss: 0.6030 - root_mean_squared_error: 0.7672 - val_loss: 0.5988 - val_root_mean_squared_error: 0.7649
Epoch 379/500
17/17 [==============================] - 0s 11ms/step - loss: 0.5989 - root_mean_squared_error: 0.7642 - val_loss: 0.6021 - val_root_mean_squared_error: 0.7666
Epoch 380/500
17/17 [==============================] - 0s 11ms/step - loss: 0.5990 - root_mean_squared_error: 0.7646 - val_loss: 0.6029 - val_root_mean_squared_error: 0.7673
Epoch 381/500
17/17 [==============================] - 0s 12ms/step - loss: 0.5991 - root_mean_squared_error: 0.7649 - val_loss: 0.6106 - val_root_mean_squared_error: 0.7723
Epoch 382/500
17/17 [==============================] - 0s 11ms/step - loss: 0.6061 - root_mean_squared_error: 0.7689 - val_loss: 0.5938 - val_ro

17/17 [==============================] - 0s 13ms/step - loss: 0.6007 - root_mean_squared_error: 0.7658 - val_loss: 0.5992 - val_root_mean_squared_error: 0.7647
Epoch 425/500
17/17 [==============================] - 0s 13ms/step - loss: 0.6035 - root_mean_squared_error: 0.7671 - val_loss: 0.6016 - val_root_mean_squared_error: 0.7667
Epoch 426/500
17/17 [==============================] - 0s 12ms/step - loss: 0.6050 - root_mean_squared_error: 0.7687 - val_loss: 0.6032 - val_root_mean_squared_error: 0.7670
Epoch 427/500
17/17 [==============================] - 0s 12ms/step - loss: 0.6076 - root_mean_squared_error: 0.7699 - val_loss: 0.6074 - val_root_mean_squared_error: 0.7698
Epoch 428/500
17/17 [==============================] - 0s 10ms/step - loss: 0.5989 - root_mean_squared_error: 0.7649 - val_loss: 0.5955 - val_root_mean_squared_error: 0.7625
Epoch 429/500
17/17 [==============================] - 0s 10ms/step - loss: 0.6059 - root_mean_squared_error: 0.7690 - val_loss: 0.5910 - val_ro

17/17 [==============================] - 0s 11ms/step - loss: 0.5916 - root_mean_squared_error: 0.7597 - val_loss: 0.5921 - val_root_mean_squared_error: 0.7607
Epoch 472/500
17/17 [==============================] - 0s 11ms/step - loss: 0.5913 - root_mean_squared_error: 0.7589 - val_loss: 0.5902 - val_root_mean_squared_error: 0.7590
Epoch 473/500
17/17 [==============================] - 0s 12ms/step - loss: 0.6012 - root_mean_squared_error: 0.7656 - val_loss: 0.6100 - val_root_mean_squared_error: 0.7713
Epoch 474/500
17/17 [==============================] - 0s 12ms/step - loss: 0.6068 - root_mean_squared_error: 0.7697 - val_loss: 0.6095 - val_root_mean_squared_error: 0.7703
Epoch 475/500
17/17 [==============================] - 0s 12ms/step - loss: 0.6001 - root_mean_squared_error: 0.7648 - val_loss: 0.5805 - val_root_mean_squared_error: 0.7519
Epoch 476/500
17/17 [==============================] - 0s 11ms/step - loss: 0.6019 - root_mean_squared_error: 0.7666 - val_loss: 0.5965 - val_ro

# Experiment 3: probabilistic Bayesian neural network

In [17]:
def create_probablistic_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # Create a probabilisticå output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # We set units=2 to learn both the mean and the variance of the Normal distribution.
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [18]:
# our loss function to compute how likely to see the true data (targets) from the estimated distribution produced by the model
def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)


num_epochs = 1000
prob_bnn_model = create_probablistic_bnn_model(train_size)
run_experiment(prob_bnn_model, negative_loglikelihood, train_dataset, test_dataset)

Start training the model...
Epoch 1/1000
17/17 [==============================] - 8s 129ms/step - loss: 84.1201 - root_mean_squared_error: 6.7469 - val_loss: 122.5071 - val_root_mean_squared_error: 6.7010
Epoch 2/1000
17/17 [==============================] - 0s 12ms/step - loss: 94.8864 - root_mean_squared_error: 6.7153 - val_loss: 38.2163 - val_root_mean_squared_error: 6.3306
Epoch 3/1000
17/17 [==============================] - 0s 12ms/step - loss: 68.8469 - root_mean_squared_error: 6.5322 - val_loss: 89.2573 - val_root_mean_squared_error: 6.5107
Epoch 4/1000
17/17 [==============================] - 0s 12ms/step - loss: 44.0936 - root_mean_squared_error: 6.4418 - val_loss: 69.5983 - val_root_mean_squared_error: 6.5640
Epoch 5/1000
17/17 [==============================] - 0s 12ms/step - loss: 45.6062 - root_mean_squared_error: 6.3880 - val_loss: 46.0047 - val_root_mean_squared_error: 6.4140
Epoch 6/1000
17/17 [==============================] - 0s 11ms/step - loss: 36.8182 - root_mean_

Epoch 48/1000
17/17 [==============================] - 0s 11ms/step - loss: 2.3619 - root_mean_squared_error: 3.8972 - val_loss: 2.3496 - val_root_mean_squared_error: 3.8065
Epoch 49/1000
17/17 [==============================] - 0s 13ms/step - loss: 2.3219 - root_mean_squared_error: 3.7278 - val_loss: 2.2586 - val_root_mean_squared_error: 3.6520
Epoch 50/1000
17/17 [==============================] - 0s 14ms/step - loss: 2.2639 - root_mean_squared_error: 3.5643 - val_loss: 2.2243 - val_root_mean_squared_error: 3.4987
Epoch 51/1000
17/17 [==============================] - 0s 12ms/step - loss: 2.2102 - root_mean_squared_error: 3.4161 - val_loss: 2.1759 - val_root_mean_squared_error: 3.3291
Epoch 52/1000
17/17 [==============================] - 0s 15ms/step - loss: 2.1908 - root_mean_squared_error: 3.3353 - val_loss: 2.1446 - val_root_mean_squared_error: 2.9804
Epoch 53/1000
17/17 [==============================] - 0s 13ms/step - loss: 2.1296 - root_mean_squared_error: 3.1872 - val_loss: 2

17/17 [==============================] - 0s 13ms/step - loss: 1.2560 - root_mean_squared_error: 1.2034 - val_loss: 1.2405 - val_root_mean_squared_error: 1.2249
Epoch 96/1000
17/17 [==============================] - 0s 13ms/step - loss: 1.2537 - root_mean_squared_error: 1.2070 - val_loss: 1.2499 - val_root_mean_squared_error: 1.1469
Epoch 97/1000
17/17 [==============================] - 0s 12ms/step - loss: 1.2659 - root_mean_squared_error: 1.2135 - val_loss: 1.2501 - val_root_mean_squared_error: 1.1798
Epoch 98/1000
17/17 [==============================] - 0s 11ms/step - loss: 1.2629 - root_mean_squared_error: 1.1969 - val_loss: 1.2480 - val_root_mean_squared_error: 1.1607
Epoch 99/1000
17/17 [==============================] - 0s 12ms/step - loss: 1.2527 - root_mean_squared_error: 1.1831 - val_loss: 1.2770 - val_root_mean_squared_error: 1.1650
Epoch 100/1000
17/17 [==============================] - 0s 12ms/step - loss: 1.2756 - root_mean_squared_error: 1.2314 - val_loss: 1.2565 - val_r

Epoch 142/1000
17/17 [==============================] - 0s 12ms/step - loss: 1.1947 - root_mean_squared_error: 1.1379 - val_loss: 1.2102 - val_root_mean_squared_error: 1.0720
Epoch 143/1000
17/17 [==============================] - 0s 14ms/step - loss: 1.2104 - root_mean_squared_error: 1.1290 - val_loss: 1.2161 - val_root_mean_squared_error: 1.1883
Epoch 144/1000
17/17 [==============================] - 0s 13ms/step - loss: 1.2151 - root_mean_squared_error: 1.1344 - val_loss: 1.2230 - val_root_mean_squared_error: 1.1480
Epoch 145/1000
17/17 [==============================] - 0s 12ms/step - loss: 1.2087 - root_mean_squared_error: 1.1291 - val_loss: 1.2317 - val_root_mean_squared_error: 1.1451
Epoch 146/1000
17/17 [==============================] - 0s 13ms/step - loss: 1.2070 - root_mean_squared_error: 1.1333 - val_loss: 1.2105 - val_root_mean_squared_error: 1.1335
Epoch 147/1000
17/17 [==============================] - 0s 13ms/step - loss: 1.2121 - root_mean_squared_error: 1.1298 - val_l

Epoch 189/1000
17/17 [==============================] - 0s 11ms/step - loss: 1.1907 - root_mean_squared_error: 1.1300 - val_loss: 1.1759 - val_root_mean_squared_error: 1.1377
Epoch 190/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1755 - root_mean_squared_error: 1.0959 - val_loss: 1.2138 - val_root_mean_squared_error: 1.0941
Epoch 191/1000
17/17 [==============================] - 0s 9ms/step - loss: 1.1816 - root_mean_squared_error: 1.1180 - val_loss: 1.2007 - val_root_mean_squared_error: 1.1351
Epoch 192/1000
17/17 [==============================] - 0s 11ms/step - loss: 1.1879 - root_mean_squared_error: 1.1035 - val_loss: 1.2097 - val_root_mean_squared_error: 1.1480
Epoch 193/1000
17/17 [==============================] - 0s 12ms/step - loss: 1.1761 - root_mean_squared_error: 1.0899 - val_loss: 1.1696 - val_root_mean_squared_error: 1.0520
Epoch 194/1000
17/17 [==============================] - 0s 11ms/step - loss: 1.1775 - root_mean_squared_error: 1.1103 - val_los

...

Epoch 800/1000
17/17 [==============================] - 0s 12ms/step - loss: 1.1486 - root_mean_squared_error: 1.0786 - val_loss: 1.1458 - val_root_mean_squared_error: 1.0263
Epoch 801/1000
17/17 [==============================] - 0s 13ms/step - loss: 1.1570 - root_mean_squared_error: 1.0675 - val_loss: 1.1566 - val_root_mean_squared_error: 1.0687
Epoch 802/1000
17/17 [==============================] - 0s 13ms/step - loss: 1.1481 - root_mean_squared_error: 1.0636 - val_loss: 1.1610 - val_root_mean_squared_error: 1.1039
Epoch 803/1000
17/17 [==============================] - 0s 14ms/step - loss: 1.1500 - root_mean_squared_error: 1.0487 - val_loss: 1.1650 - val_root_mean_squared_error: 1.0398
Epoch 804/1000
17/17 [==============================] - 0s 13ms/step - loss: 1.1661 - root_mean_squared_error: 1.0761 - val_loss: 1.1467 - val_root_mean_squared_error: 1.1050
Epoch 805/1000
17/17 [==============================] - 0s 13ms/step - loss: 1.1508 - root_mean_squared_error: 1.0611 - val_l

Epoch 847/1000
17/17 [==============================] - 0s 14ms/step - loss: 1.1487 - root_mean_squared_error: 1.0592 - val_loss: 1.1463 - val_root_mean_squared_error: 1.0747
Epoch 848/1000
17/17 [==============================] - 0s 14ms/step - loss: 1.1546 - root_mean_squared_error: 1.0698 - val_loss: 1.1738 - val_root_mean_squared_error: 1.0456
Epoch 849/1000
17/17 [==============================] - 0s 14ms/step - loss: 1.1485 - root_mean_squared_error: 1.0747 - val_loss: 1.1526 - val_root_mean_squared_error: 1.0842
Epoch 850/1000
17/17 [==============================] - 0s 13ms/step - loss: 1.1491 - root_mean_squared_error: 1.0560 - val_loss: 1.1538 - val_root_mean_squared_error: 1.0387
Epoch 851/1000
17/17 [==============================] - 0s 13ms/step - loss: 1.1484 - root_mean_squared_error: 1.0672 - val_loss: 1.1637 - val_root_mean_squared_error: 1.0706
Epoch 852/1000
17/17 [==============================] - 0s 12ms/step - loss: 1.1608 - root_mean_squared_error: 1.0805 - val_l

Epoch 894/1000
17/17 [==============================] - 0s 14ms/step - loss: 1.1526 - root_mean_squared_error: 1.0579 - val_loss: 1.1500 - val_root_mean_squared_error: 1.1301
Epoch 895/1000
17/17 [==============================] - 0s 13ms/step - loss: 1.1536 - root_mean_squared_error: 1.0888 - val_loss: 1.1450 - val_root_mean_squared_error: 1.0617
Epoch 896/1000
17/17 [==============================] - 0s 13ms/step - loss: 1.1552 - root_mean_squared_error: 1.0825 - val_loss: 1.1619 - val_root_mean_squared_error: 1.0802
Epoch 897/1000
17/17 [==============================] - 0s 14ms/step - loss: 1.1469 - root_mean_squared_error: 1.0801 - val_loss: 1.1563 - val_root_mean_squared_error: 1.1037
Epoch 898/1000
17/17 [==============================] - 0s 11ms/step - loss: 1.1530 - root_mean_squared_error: 1.0630 - val_loss: 1.1481 - val_root_mean_squared_error: 1.0540
Epoch 899/1000
17/17 [==============================] - 0s 12ms/step - loss: 1.1529 - root_mean_squared_error: 1.0652 - val_l

Epoch 941/1000
17/17 [==============================] - 0s 13ms/step - loss: 1.1476 - root_mean_squared_error: 1.0451 - val_loss: 1.1488 - val_root_mean_squared_error: 1.0913
Epoch 942/1000
17/17 [==============================] - 0s 14ms/step - loss: 1.1453 - root_mean_squared_error: 1.0583 - val_loss: 1.1559 - val_root_mean_squared_error: 1.1369
Epoch 943/1000
17/17 [==============================] - 0s 13ms/step - loss: 1.1495 - root_mean_squared_error: 1.0685 - val_loss: 1.1540 - val_root_mean_squared_error: 1.0760
Epoch 944/1000
17/17 [==============================] - 0s 14ms/step - loss: 1.1528 - root_mean_squared_error: 1.0889 - val_loss: 1.1588 - val_root_mean_squared_error: 1.0479
Epoch 945/1000
17/17 [==============================] - 0s 12ms/step - loss: 1.1452 - root_mean_squared_error: 1.0644 - val_loss: 1.1682 - val_root_mean_squared_error: 1.0951
Epoch 946/1000
17/17 [==============================] - 0s 13ms/step - loss: 1.1474 - root_mean_squared_error: 1.0698 - val_l

Epoch 988/1000
17/17 [==============================] - 0s 12ms/step - loss: 1.1413 - root_mean_squared_error: 1.0423 - val_loss: 1.1427 - val_root_mean_squared_error: 1.0222
Epoch 989/1000
17/17 [==============================] - 0s 12ms/step - loss: 1.1487 - root_mean_squared_error: 1.0586 - val_loss: 1.1496 - val_root_mean_squared_error: 1.0828
Epoch 990/1000
17/17 [==============================] - 0s 12ms/step - loss: 1.1438 - root_mean_squared_error: 1.0570 - val_loss: 1.1512 - val_root_mean_squared_error: 1.1172
Epoch 991/1000
17/17 [==============================] - 0s 13ms/step - loss: 1.1488 - root_mean_squared_error: 1.0724 - val_loss: 1.1514 - val_root_mean_squared_error: 1.0590
Epoch 992/1000
17/17 [==============================] - 0s 13ms/step - loss: 1.1515 - root_mean_squared_error: 1.0576 - val_loss: 1.1508 - val_root_mean_squared_error: 1.0568
Epoch 993/1000
17/17 [==============================] - 0s 13ms/step - loss: 1.1491 - root_mean_squared_error: 1.0601 - val_l

In [19]:
#produce an output from the model given the test examples
prediction_distribution = prob_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

for idx in range(sample):
    print(
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {targets[idx]}"
    )

Prediction mean: 5.5, stddev: 0.76, 95% CI: [6.99 - 4.02] - Actual: 5.0
Prediction mean: 6.02, stddev: 0.77, 95% CI: [7.52 - 4.52] - Actual: 5.0
Prediction mean: 6.19, stddev: 0.79, 95% CI: [7.74 - 4.64] - Actual: 5.0
Prediction mean: 6.07, stddev: 0.8, 95% CI: [7.63 - 4.51] - Actual: 6.0
Prediction mean: 5.23, stddev: 0.7, 95% CI: [6.61 - 3.86] - Actual: 5.0
Prediction mean: 5.99, stddev: 0.75, 95% CI: [7.46 - 4.52] - Actual: 7.0
Prediction mean: 5.37, stddev: 0.75, 95% CI: [6.83 - 3.9] - Actual: 5.0
Prediction mean: 5.89, stddev: 0.75, 95% CI: [7.36 - 4.42] - Actual: 5.0
Prediction mean: 6.41, stddev: 0.81, 95% CI: [7.99 - 4.82] - Actual: 6.0
Prediction mean: 6.1, stddev: 0.75, 95% CI: [7.58 - 4.62] - Actual: 7.0
